In [ ]:
import open3d as o3d
import numpy as np
import laspy as lp
import open3d.visualization as viz
import time
import matplotlib.pyplot as plt

In [ ]:
# example code on how to read a LAS file
las = lp.read('./data/Pole1.las')
points = np.vstack((las.x, las.y, las.z)).transpose()

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(points)

print(f'sample: {points[0:5]}')
print(f'shape: {points.shape}')
print(f'min_x = {las.header.x_min}, max_x = {las.header.x_max}')
print(f'min_y = {las.header.y_min}, max_y = {las.header.y_max}')
print(f'min_z = {las.header.z_min}, max_z = {las.header.z_max}')

In [ ]:
def visualize(geoms, capture_filename=''):
    '''
        Helper function to run the Open3D visualizer
        Usage: 

        From the pcd variable above, you can call visualize with:
        visualize([pcd])


        Axis:   x = red
                y = green
                z = blue
    '''
    v = viz.Visualizer()
    v.create_window()
    opt = v.get_render_option()
    opt.show_coordinate_frame = True
    
    for g in geoms:
        v.add_geometry(g)
    
    ctr = v.get_view_control()
    # assumes default of 1920x1080 window
    # comment out these two lines if they cause display issues
    camera_params = o3d.io.read_pinhole_camera_parameters('./camera_trajectory.json')
    ctr.convert_from_pinhole_camera_parameters(camera_params)

    if capture_filename != '':
        time.sleep(1)
        v.capture_screen_image(capture_filename, True)
    else:
        # press "h" for help when in the visualizer for the commands
        v.run()
    v.destroy_window()

In [ ]:
'''
####################################
    Write your solution here
####################################
'''
def solve(filepath: str) -> list[o3d.geometry.PointCloud]:
    las = lp.read(filepath)
    points = np.vstack((las.x, las.y, las.z)).transpose()

    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    
    range_z = las.header.z_max - las.header.z_min
    threshold = 0.1
    z_max_threshold = (range_z * threshold) + las.header.z_min

    ground_points_idx = np.transpose(np.where(points[:, 2] <= z_max_threshold))

    # RGB, colour everything red first
    pcd.paint_uniform_color([1,0,0])

    # colour the ground points dark green
    ground_points = pcd.select_by_index(ground_points_idx)
    ground_points.paint_uniform_color([0.047,0.117,0.050])

    return [pcd, ground_points]

In [ ]:
num_samples = 5

for n in range(num_samples + 1):
    geoms = solve(f'data/Pole{n}.las')
    visualize(geoms, f'naive_ground{n}.png')
